In [1]:
!pip3 install sqlalchemy PyMySQL

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import sqlalchemy as sqlal
from pymysql import OperationalError

from utils_misc import pickle_load_or_create, pickle_save
from utils_cv2 import get_frame, show_frame_and_await_key, label_frames_from_df

Defaulting to user installation because normal site-packages is not writeable


In [3]:
HOST = '127.0.0.1'
PORT = '3377'
DATABASE = 'judge'
USERNAME = 'root'
PASSWORD = 'root'
DATABASE_CONNECTION=f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

In [4]:
def get_connection():
    engine = sqlal.create_engine(DATABASE_CONNECTION)#
    return engine.connect()
con = get_connection()

In [5]:
def executeSqlScriptsFromFile(filename):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')

    # Execute every command from the input file
    for command in sqlCommands:
        # This will skip and report errors
        # For example, if the tables do not yet exist, this will skip over
        # the DROP TABLE commands
        try:
            con.execute(sqlal.text(command))
        except OperationalError as msg:
            print("Command skipped: ", msg)

In [6]:
def initDatabase():
    try:
        executeSqlScriptsFromFile('../db/init.sql')
    except Exception as e:
        print(e)

In [7]:
def insert_folders():
    qry = "INSERT INTO Folders (path) VALUES ('videos'), ('belgium/bk'), ('belgium/pk')"
    con.execute(sqlal.text(qry))
    con.commit()

In [8]:
def add_video(folder_id, name, training, obstruction, quality, type, dimensionX, dimensionY):
    command = sqlal.text(f"""
    INSERT INTO Videos (folderID, name, training, obstruction, quality, type, dimensionX, dimensionY) 
    VALUES ({folder_id}, \'{name}\', {training}, {obstruction}, {quality}, {type}, {dimensionX}, {dimensionY})
    """)
    con.execute(command)
    con.commit()

In [9]:
video_names_train = [
    '20240201_atelier_001.mp4',
    '20240201_atelier_002.mp4',
    '20240201_atelier_003.mp4',
    '20240201_atelier_004.mp4',
    '20240201_atelier_005.mp4',
    '20240209_atelier_006.mp4',
    '20240209_atelier_007.mp4',
    '20240209_atelier_008.mp4',
]

video_names_test = [
    '20240209_atelier_009.mp4',
]

def insert_videos():
    for vid in video_names_train:
        add_video(1, vid, 1, 0, 70, 2, 1920, 1080)
    for vid in video_names_test:
        add_video(1, vid, 0, 0, 70, 2, 1920, 1080)

In [10]:
def get_video_name_id_dict():
    qry = sqlal.text("SELECT name, videoID FROM Videos")
    dict_videos = {}
    for idx, row in pd.read_sql_query(qry, con).iterrows():
        dict_videos[row['name']] = row['videoID']
    return dict_videos

In [11]:
def add_borderlabel(videoID, frameNR, label, manual=1):
    command = sqlal.text(f"""
    INSERT INTO FrameLabels values ({videoID}, {frameNR}, {label}, {manual})
    """)
    con.execute(command)
    con.commit()

In [12]:
def insert_framelabels():
    df_labels = pickle_load_or_create('df_video_border_labels_seq', lambda: pd.DataFrame())
    video_dict = get_video_name_id_dict()
    for _, row in df_labels.iterrows():
        vid_id = video_dict[row['path']]
        add_borderlabel(vid_id, row['frame'], row['border'])

In [13]:
def refreshDatabase():
    initDatabase()
    insert_folders()
    insert_videos()
    insert_framelabels()

In [14]:
def label_exists(vid_id, frame_nr, label):
    qry = f"SELECT COUNT(*) as count FROM FrameLabels WHERE videoID = {vid_id} AND frameNr = {frame_nr} AND label = {label};"
    return (pd.read_sql_query(qry, con) > 0)['count'][0]

In [15]:
def query_framelabels():
    qry=sqlal.text(f"""
    SELECT DISTINCT * FROM FrameLabels;
    """)
    return pd.read_sql_query(qry, con)

In [16]:
refreshDatabase()

In [17]:
query_framelabels()

,videoID,frameNr,label,manual_insert
0,1,0,9,1
1,1,1,9,1
2,1,2,9,1
3,1,3,9,1
4,1,4,9,1
...,...,...,...,...
2859,2,763,2,1
2860,2,764,2,1
2861,2,765,2,1
2862,2,766,2,1
